## Создаём новое приложение

1. Необходимо перейти в "Управление" (панель управления приложениями)
![Apps management](./vk_screens/1.jpg)

2. Создать новое приложение
![New app](./vk_screens/2.jpg)

3. В открывшейся форме необходимо указать название приложения и выбрать тип "Standalone-приложение"
![App form](./vk_screens/3.jpg)

4. После создания необходимо перейти в "Настройки"

5. Включить приложение и Open API

6. В пункте "Адрес приложения" указать значение `http://localhost`

7. В пункте "Базовый домен" указать `localhost`
![Settings](./vk_screens/4.jpg)

8. Скопировать id клиента

9. Сформировать ссылку для получения ключа доступа (токена)
`https://oauth.vk.com/authorize?client_id=...&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=stats,offline&response_type=token`в параметр `client_id` подставить скопированный id Вашего приложения.

    Не забудьте указать необходимые scope https://vk.com/dev/permissions

    Подробнее про получение токена здесь: https://dev.vk.com/api/access-token/implicit-flow-user

10. При переходе по ссылке, откроется окно подтверждения, необходимо нажать кнопку "Разрешить"
![Confirm](./vk_screens/5.jpg)

11. Затем в адресной строке открывшейся страницы скопировать значение параметра access_token
![Token](./vk_screens/6.jpg)

## Что нужно знать перед началом работы с VK API

1. [Синтаксис любого запроса](https://vk.com/dev/api_requests)

2. [Методы API VK](https://vk.com/dev/methods)

3. [Версии API VK](https://vk.com/dev/versions)

4. [Ограничения](https://vk.com/dev/api_requests?f=3.%20%D0%9E%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%20%D0%B8%20%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8)

5. [Получение ключа доступа (Implicit flow)](https://dev.vk.com/api/access-token/implicit-flow-user)

### Читаем токен из файла. В файлах с кодом приватные данные (ключи доступа, пароли) и настройки хранить не принято

In [3]:
with open('token.txt', 'r') as token_file:
    token = token_file.readline()

### Получим базовую информацию о пользователе при помощи [users.get](https://vk.com/dev/users.get)

In [4]:
import os
import time
from pprint import pprint

import pandas as pd
import requests



def get_user_data(user_ids, token, version):
    method_name = f'https://api.vk.com/method/users.get'
    params = {'user_ids': user_ids, 'access_token': token, 'v': version}
    response = requests.get(method_name, params=params)
    data = response.json()
    return data


data = get_user_data(1, token, '5.131')
pprint(data)

{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'}]}


### Получим дополнительно еще какие-нибудь поля

In [5]:
def get_user_data_extended(user_ids, fields, token, version):
    method_name = 'https://api.vk.com/method/users.get'
    params = {'user_ids': user_ids, 'fields': fields, 'access_token': token, 'v': version}
    response = requests.get(method_name, params=params)
    data = response.json()
    return data

data = get_user_data_extended(1, 'bdate,city,followers_count', token, '5.131')
pprint(data)

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'city': {'id': 2, 'title': 'Санкт-Петербург'},
               'first_name': 'Павел',
               'followers_count': 4823561,
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'}]}


### Напишем функцию, которая будет находить группы по поисковому запросу при помощи метода [groups.search](https://vk.com/dev/groups.search)

In [6]:
def search_groups(q, sort, count, token, version):
    url = 'https://api.vk.com/method/groups.search'
    params = {'q': q, 'sort': sort, 'count': count, 'access_token': token, 'v': version}
    response = requests.get(url, params=params)
    data = response.json()
    return data['response']


groups = search_groups('авто', 6, 100, token, '5.131')

## но для красивого вывода информации вместо print() и pprint() используем модуль pandas

pd.DataFrame(groups['items'])

,id,name,screen_name,is_closed,type,is_admin,is_member,is_advertiser,photo_50,photo_100,photo_200
0,31128331,Про Авто,autoblogvk,0,page,0,0,0,https://sun1-57.userapi.com/s/v1/if1/W1Vi_sbnh...,https://sun1-57.userapi.com/s/v1/if1/6y27ftTvk...,https://sun1-57.userapi.com/s/v1/if1/v8lL3ubrP...
1,48990018,АВТО КЛУБ,klubauto,0,page,0,0,0,https://sun1-54.userapi.com/s/v1/ig2/yVKsbT8hx...,https://sun1-54.userapi.com/s/v1/ig2/T-JYx17nZ...,https://sun1-54.userapi.com/s/v1/ig2/zWpuhJI7r...
2,148411246,АВТО ЛАЙФХАК,autol,0,page,0,0,0,https://sun1-18.userapi.com/s/v1/if1/fAFlm5RrK...,https://sun1-18.userapi.com/s/v1/if1/4QrFsagvU...,https://sun1-18.userapi.com/s/v1/if1/poPqqkR0a...
3,90060110,АВТО,autoclub_i,0,page,0,0,0,https://sun1-19.userapi.com/s/v1/if1/w3TA1cxrw...,https://sun1-19.userapi.com/s/v1/if1/O6xExEEkm...,https://sun1-19.userapi.com/s/v1/if1/o3T03j67M...
4,43658512,"АВТО, МАШИНЫ, ТАЧКИ",isupercars,0,page,0,0,0,https://sun1-98.userapi.com/s/v1/ig2/gycnY1fU0...,https://sun1-98.userapi.com/s/v1/ig2/SrTlS_Iue...,https://sun1-98.userapi.com/s/v1/ig2/myhN9q6ga...
...,...,...,...,...,...,...,...,...,...,...,...
95,167219324,АВТО СПЕЦ,topautospec,0,page,0,0,0,https://sun1-14.userapi.com/s/v1/ig2/s1FPoFAFF...,https://sun1-14.userapi.com/s/v1/ig2/amR7Ej7cx...,https://sun1-14.userapi.com/s/v1/ig2/BIOQ3auG4...
96,49270909,БПАН МУЗЫКА / БЕЗ ПОСАДКИ - АВТО.NET,bpan.music,0,page,0,0,0,https://sun1-98.userapi.com/s/v1/if1/-F0FwjrrF...,https://sun1-98.userapi.com/s/v1/if1/k2gA-ZfOw...,https://sun1-98.userapi.com/s/v1/if1/Y_P50neyD...
97,39464472,Авто приколы,avtoumar,0,page,0,0,0,https://sun1-19.userapi.com/s/v1/ig2/uSlx-2PpC...,https://sun1-19.userapi.com/s/v1/ig2/uJbHijCx5...,https://sun1-19.userapi.com/s/v1/ig2/otz5K6T5C...
98,109329578,БЕЗ ПОСАДКИ-АВТО.NET | БПAN,club109329578,0,page,0,0,0,https://sun1-23.userapi.com/s/v1/ig2/ySZMbqQK7...,https://sun1-23.userapi.com/s/v1/ig2/EzZUrZwtR...,https://sun1-23.userapi.com/s/v1/ig2/ZSqV_ib6C...


### Если строим какую-то сложную логику взаимодействия с API, то логично будет инкапсулировать весь нужный функционал в класс. Какие нам нужны данные, чтобы инициализировать класс?

In [7]:
# токен и версия могут быть разные в разных экзмеплярах
# базовый URL будет всегда один, в инициализации он не нужен
class VkApiHandler:

    base_url = 'https://api.vk.com/method/'

    def __init__(self, access_token, version='5.131'):
        self.params = {
            'access_token': access_token,
            'v': version
        }

### Объединим ранее написанные функции в класс и добавим ещё один метод для поиска новостей

In [8]:
class VkApiHandler:

    base_url = 'https://api.vk.com/method/'

    def __init__(self, access_token, version='5.131'):
        self.params = {
            'access_token': access_token,
            'v': version
        }

    def get_user_data(self, user_ids):
        method_name = f'{self.base_url}users.get'
        params = {'user_ids': user_ids}.update(self.params)
        response = requests.get(method_name, params=params)
        data = response.json()
        return data

    def get_user_data_extended(self, user_ids, fields):
        method_name = f'{self.base_url}users.get'
        params = {'user_ids': user_ids, 'fields': fields, **self.params}
        response = requests.get(method_name, params=params)
        data = response.json()
        return data

    def search_groups(self, q, sort, count):
        url = self.base_url + 'groups.search'
        params = {'q': q, 'sort': sort, 'count': count, **self.params}
        response = requests.get(url, params=params)
        data = response.json()
        return data['response']

    def search_news(self, q):
        url = self.base_url + 'newsfeed.search'
        params = {'q': q, 'count': 200, **self.params}

        news_frame = pd.DataFrame()

        while True:
            #добавляем тайм-аут, чтобы запросы не блокировались
            time.sleep(0.33)
            response = requests.get(url, params=params)
            data = response.json()
            news_frame = pd.concat([news_frame, pd.DataFrame(data['response']['items'])])
            if 'next_from' in data['response']:
                params.update({'start_from': data['response']['next_from']})
            else:
                break
        return news_frame


if __name__ == "__main__":
    vk_token = os.getenv('VK_API_TOKEN')
    version = os.getenv('VERSION')

    vk = VkApiHandler(vk_token, version)
    news = vk.search_news('python')

### вынесем токен и версию в переменные окружения

![ENV VARS](./vk_screens/7.jpg)

### а затем подгрузим их при помощи модуля python-dotenv

```python
import os
import time
from dotenv import load_dotenv

...

load_dotenv()

...

if __name__ == "__main__":
    vk_token = os.getenv('VK_API_TOKEN')
    version = os.getenv('VERSION')

    vk = VkApiHandler(vk_token, version)
    vk.search_news('авто')
```